# Parte B. 
### Elaboração de um conjunto de scripts e funções em Python, NumPy, SciPy e Scikit-learn para realizar as tarefas de Aprendizagem Computacional e Avaliação

In [245]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

# importar livraria que contem as funcoes das features
import featuresFunctions
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.metrics import recall_score, precision_score, f1_score
from sklearn.neighbors import KNeighborsClassifier

# ex 4
from sklearn.neural_network import MLPClassifier

import warnings
warnings.filterwarnings("ignore")

___

# 0. Utility functions
### 0.1. Loading human activities dataset

In [200]:
def load_human_activities_dataset():
    loc = []

    for i in range (0,15):
        loc.append("FORTH_TRACE_DATASET-master/part" + str(i) + "/part" + str(i) + "dev2.csv")

    array = []
    for file in loc:
        df = pd.read_csv(file, sep=',', header=None)
        array.append(df.to_numpy())

    array = np.concatenate(array)
    #print(array.shape)

    return array

human_activities_data = load_human_activities_dataset()
human_activities_input = human_activities_data[:, :11]
human_activities_target = human_activities_data[:, -1]

dummy = human_activities_data[:6]
dummy_input = dummy[:, :11]
dummy_target = dummy[:, -1]

human_activities_input

array([[2.0000e+00, 3.0317e+00, 9.0450e+00, ..., 6.8849e-01, 1.2654e+00,
        6.3629e+01],
       [2.0000e+00, 2.9591e+00, 9.0436e+00, ..., 6.8849e-01, 1.2741e+00,
        8.3160e+01],
       [2.0000e+00, 2.9465e+00, 9.0545e+00, ..., 6.7857e-01, 1.2675e+00,
        1.0269e+02],
       ...,
       [2.0000e+00, 1.7283e+00, 9.5641e+00, ..., 9.2857e-01, 1.3531e+00,
        1.0420e+06],
       [2.0000e+00, 1.7285e+00, 9.5646e+00, ..., 9.0675e-01, 1.3377e+00,
        1.0420e+06],
       [2.0000e+00, 1.6802e+00, 9.4907e+00, ..., 8.6310e-01, 1.3180e+00,
        1.0420e+06]])

### 0.2. Loading Iris dataset

150 instances <br>
4 atributes:
1. sepal length in cm
2. sepal width in cm
3. petal length in cm
4. petal width in cm

3 classes:
- Iris Setosa
- Iris Versicolour
- Iris Virginica

In [209]:
from sklearn.datasets import load_iris

iris_data = load_iris()

df = pd.DataFrame(iris_data.data, columns = iris_data.feature_names)
df['target'] = iris_data.target
print(df)
iris_df = df
iris_input = iris_data.data
iris_target = iris_data.target

iris_input

     sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                  5.1               3.5                1.4               0.2   
1                  4.9               3.0                1.4               0.2   
2                  4.7               3.2                1.3               0.2   
3                  4.6               3.1                1.5               0.2   
4                  5.0               3.6                1.4               0.2   
..                 ...               ...                ...               ...   
145                6.7               3.0                5.2               2.3   
146                6.3               2.5                5.0               1.9   
147                6.5               3.0                5.2               2.0   
148                6.2               3.4                5.4               2.3   
149                5.9               3.0                5.1               1.8   

     target  
0         0  

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3

___

# 1. Data splitting e métricas de exactidão em machine learning
### 1.1. Usando o scikit-learn, desenvolva um conjunto de funções para **data splitting** usando dois cenários:

### 1.1.1. Train-Test (TT) e Train-Validation-Test data split

Divides x (input) and y (target) in: <br>
- x_train = x * test_perc   
- x_val = x * val_perc         
- x_test = x - x_train - x_val
- y_train = y * test_perc     
- y_val = y * val_perc
- y_test = y - y_train - y_val

In [82]:
def tt(x, y, test_perc):
    test_size = test_perc * 0.01
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size, random_state=1)

    return x_train, x_test, y_train, y_test


def tvt(x, y, val_perc, test_perc):
    test_size = test_perc * 0.01
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size, random_state=1)

    # 80 train, 20 test -> 60 train, 20 validation, 20 test
    val_size = test_perc * 0.01
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=val_size, random_state=1) # 0.25 x 0.8 = 0.2

    return x_train, x_val, x_test, y_train, y_val, y_test


### 1.1.2. K-fold data split

K-folds cross-validator. Returns k indexes of training and validating (min - 2. max - samples)

In [196]:
def k_fold_indexes(data_input, splits):
    kf = KFold(n_splits=splits)

    x_indexes_train=[]
    x_indexes_test=[]
    for i, (train_index, test_index) in enumerate(kf.split(data_input)):
        x_indexes_train.append(train_index)
        x_indexes_test.append(test_index)

        print(train_index, test_index)

    return x_indexes_train, x_indexes_test


In [197]:
### TEST CODE

# TVT
#dummy_input_train, dummy_input_val, dummy_input_test, dummy_target_train, dummy_target_val, dummy_target_test = tvt(human_activities_input, human_activities_target, 20, 20)

#print(human_activities_input.shape, human_activities_target.shape, dummy_input_train.shape, dummy_input_val.shape, dummy_input_test.shape, dummy_target_train.shape, dummy_target_val.shape, dummy_target_test.shape)

# K-FOLD
dummy_train_indexes, dummy_test_indexes = k_fold_indexes(dummy_target, 5)
print("---")

def k_fold_arrays(data_input, splits):
    data_input_train = []
    data_input_test = []

    train_indexes, test_indexes = k_fold_indexes(data_input, splits)

    print("---")
    for i in train_indexes:
        #print(i)
        
        data_input_train.append(data_input[i])
        data_target_train.append(data_target[i])

        #print(data_input_train, "--\n", data_target_train)

    print("---")
    for j in test_indexes:
        print(j)
        
        data_input_test.append(data_input[j])
        data_target_test.append(data_target[j])

        print( "--\n", data_target_test)



#dummy_input

#k_fold_arrays(dummy_input, dummy_target, 5)

#dummy_input_by_indexes = dummy_input[dummy_train_indexes[0]]
#dummy_input_by_indexes




#recall, precision, f1 = exact_methods(dummy_real, dummy_pred, 'macro')
#print(recall, precision, f1) # com average = 'micro' os resultados sao iguais


[2 3 4 5] [0 1]
[0 1 3 4 5] [2]
[0 1 2 4 5] [3]
[0 1 2 3 5] [4]
[0 1 2 3 4] [5]
---


### 1.2. Usando o scikit-learn, desenvolva um conjunto de funções para cálculo de **métricas de exactidação**, nomeadamente as seguintes:

### 1.1.2. Matriz de confusão
### 1.2.2. Recall
### 1.2.3. Precision
### 1.2.4. F1-score

In [256]:
#from sklearn.metrics import confusion_matrix, plot_confusion_matrix
#from sklearn.metrics import recall_score, precision_score, f1_score

def confusion(knn, y_test, y_pred):
    cm = confusion_matrix(y_test, y_pred)
    matrix = plot_confusion_matrix(knn, x_test, y_test, cmap=plt.cm.Blues)
    matrix.ax_.set_title('Confusion Matrix')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.gcf().axes[0]
    plt.gcf().axes[1]
    return cm, plt

def exact_methods(y_true, y_pred, av):
    '''
    av = micro
    av = macro
    av = binary
    av = samples
    av = weighted
    '''
    recall = recall_score(y_true, y_pred, average=av) # average=?
    precision = precision_score(y_true, y_pred, average=av)
    f1 = f1_score(y_true, y_pred, average=av)

    print("Recall: ", recall)
    print("Precision: ", precision)
    print("F1 score: ", f1)

    return recall, precision, f1


# 2. Experiências iniciais com um classificador simples, i.e., k-Nearest Neighbours (kNN)

### 2.1. Usando:
- as funções anteriores
- o algoritmo kNN
- o dataset Iris

avalie a capacidade de previsão do algoritmo neste dataset, com **k = 1** e restantes parâmetros por omissão (e.g., métrica de distância, etc.), e usando todo o conjunto de features, nos seguintes cenários:

### 2.1.1. Train-only, TT 70-30 e 10x10-fold cross-validation (10CV)

In [242]:
input_train, input_test, target_train, target_test = tt(iris_input, iris_target, 30)

knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(input_train, target_train)
target_pred = knn.predict(input_test)

print(target_test)
print(target_pred)

recall, precision, f1 = exact_methods(target_test, target_pred, None)




[0 1 1 0 2 1 2 0 0 2 1 0 2 1 1 0 1 1 0 0 1 1 1 0 2 1 0 0 1 2 1 2 1 2 2 0 1
 0 1 2 2 0 2 2 1]
[0 1 1 0 2 1 2 0 0 2 1 0 2 1 1 0 1 1 0 0 1 1 1 0 2 1 0 0 1 2 1 2 1 2 2 0 1
 0 1 2 2 0 1 2 1]
Recall:  [1.         1.         0.92307692]
Precision:  [1.         0.94736842 1.        ]
F1 score:  [1.         0.97297297 0.96      ]


In [10]:
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = tt(X, y, 0.5)
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, y_train) #train
y_pred = knn.predict(X_test)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
acc = knn.score(X_test, y_test) #test

from sklearn.model_selection import cross_val_score

knn_cv = KNeighborsClassifier(n_neighbors=6)
#train model with cv of 5 
cv_scores = cross_val_score(knn_cv, X, y, cv=10)
#print each cv score (accuracy) and average them
print(cv_scores)
print('cv_scores mean:{}'.format(np.mean(cv_scores)))

[1.         0.93333333 1.         1.         0.86666667 0.93333333
 0.93333333 1.         1.         1.        ]
cv_scores mean:0.9666666666666668


In [ ]:
neighbours = np.linspace(1, 15, 8)

for n in neighbours:
    knn = KNeighborsClassifier(n_neighbors=int(n))
    knn.fit(X, y)
    

In [ ]:
neigh = np.linspace(1, 15, 8)
print(neigh)

# TT, TVT e nCV

In [ ]:
#def tt(X, y, test_size):

def cross_validation(train_X, train_y, num_folds=10, k=1):
    dataset = list()
    dataset_split = list()
    val_acc = list()
    
    for i in range(len(train_X)):
        data = np.append(train_X[i],train_y[i])
        dataset.append(data)
    
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / num_folds)
    
    for i in range(num_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
        
    for folds in dataset_split:
        train_set= folds
        train_set = np.array(train_set)
        test_set = list()
        for row in folds:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        test_set = np.array(test_set)
        train_x = train_set[:, :-1]
        train_y = train_set[:,-1]
        test_x = test_set[:, :-1]
        predicted = predict(train_x,train_y, test_x, k)
        actual = [row[-1] for row in fold]
        accuracy = compute_accuracy(actual, predicted)
        val_acc.append(accuracy)
        
    val_acc_var = statistics.variance(val_acc)
    vall_acc = sum(val_acc)/len(val_acc)

    return vall_acc, val_acc_var

# Sobre overfitting

Overfitting refers to an unwanted behavior of a machine learning algorithm used for predictive modeling.

It is the case where model performance on the training dataset is improved at the cost of worse performance on data not seen during training, such as a holdout test dataset or new data.

We can identify if a machine learning model has overfit by first evaluating the model on the training dataset and then evaluating the same model on a holdout test dataset.

If the performance of the model on the training dataset is significantly better than the performance on the test dataset, then the model may have overfit the training dataset.


## Usar as features dadas pelo Relief

In [ ]:
neighbours = 5
x_relief = relief(X, y, neighbours, 10) # TODO qual o numero de features que se quer
# TODO tem que se alterar o algoritmo de relief?
print(x_relief.shape, y.shape)

X_train, X_test, y_train, y_test = train_test_split(x_relief, y, test_size=0.3, random_state=1)

knn = KNeighborsClassifier(n_neighbors=2)
knn.fit(X_train, y_train) #train
y_pred = knn.predict(X_test)

f1_score(y_pred, y_test, average=None)

In [ ]:
error_rate = []# Will take some time
for i in range(1, X_train.shape[1]+1):
    print(i)
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train,y_train)
    pred_i = knn.predict(X_test)
    error_rate.append(np.mean(pred_i != y_test))

plt.figure(figsize=(10,6))
plt.plot(range(1,X_train.shape[1]+1),error_rate,color='blue', linestyle='dashed', marker='o',
 markerfacecolor='red', markersize=10)
plt.title('Error Rate vs. K Value')
plt.xlabel('K')
plt.ylabel('Error Rate')

# 3. Análise ao *dataset* original

In [ ]:
loc = []

for i in range (0,15):
    loc.append("FORTH_TRACE_DATASET-master/part" + str(i) + "/part" + str(i) + "dev2.csv")

array = []
for file in loc:
    df = pd.read_csv(file, sep=',', header=None)
    array.append(df.to_numpy())

array = np.concatenate(array)
print(array.shape)

___

# 4. Repita o ponto 2 no dataset de actividades humanos utilizado neste trabalho, usando data splitting **TVT** (apenas TVT; CV não é para fazer), uma **rede neuronal feedforward** (MLP) com **3 camadas**, **número variável de neurónios na camada escondida**, função de **activação logística** em todos os neurónios, **batch learning** e as features seleccionadas nas alíneas anteriores:

### 4.1. Com velocidade de aprendizagem fixo

In [264]:
input_data, target_data = iris_input, iris_target

input_train, input_val, input_test, target_train, target_val, target_test = tvt(input_data, target_data, 30, 30)

feedforward = MLPClassifier(learning_rate='adaptive', activation='logistic', batch_size='lbfgs', solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, ), random_state=1)

feedforward.fit(input_train, target_train)

print(target_test)
target_pred = feedforward.predict(input_test)
print(target_pred)

recall, precision, f1 = exact_methods(target_test, target_pred, None)



[0 1 1 0 2 1 2 0 0 2 1 0 2 1 1 0 1 1 0 0 1 1 1 0 2 1 0 0 1 2 1 2 1 2 2 0 1
 0 1 2 2 0 2 2 1]
[0 1 1 0 2 1 2 0 0 2 1 0 2 1 1 0 1 2 0 0 1 2 2 0 2 1 0 0 1 2 1 2 1 2 2 0 1
 0 1 2 2 0 1 2 1]
Recall:  [1.         0.83333333 0.92307692]
Precision:  [1.     0.9375 0.8   ]
F1 score:  [1.         0.88235294 0.85714286]
